### Import Dependencies

In [1]:
from mediapipe import solutions as mp
import cv2
from google.protobuf.json_format import MessageToDict
import time
import os

### Load Fingers Images 

- These finger images will be shown on the webcam according to the prediction made. 

In [2]:
# Load finger images 
folder_path = "finger_images"
overlay_list = []

for img_path in os.listdir(folder_path):
    image = cv2.imread(f"{folder_path}/{img_path}")
    print(f"{folder_path}/{img_path}")
    overlay_list.append(image)
print(len(overlay_list))

finger_images/finger_0.png
finger_images/finger_1.png
finger_images/finger_2.png
finger_images/finger_3.png
finger_images/finger_4.png
finger_images/finger_5.png
6


### Finger Count Detector

To connect your phone camera:
- Install `DroidCam` App on your phone and `DroidCam Client` on your laptop. 
- Put the IP address in the `cv2.VideoCapture()` below.
- Or use other phone camera IP service provider.

In [3]:
# Using phone camera ip with DroidCam
# Webcam is 0
cap = cv2.VideoCapture("http://192.168.100.6:4747/mjpegfeed?640x480")
tip_id = [4, 8, 12, 16, 20]
pTime = 0

with mp.hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:

    while True:
        success, frame = cap.read()
        lm_list = []

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = hands.process(image)

        if results.multi_hand_landmarks:
            for hand_landmark in results.multi_hand_landmarks:
                for idx, lm in enumerate(hand_landmark.landmark):
                    h, w, c = image.shape
                    cx, cy = int(lm.x*w), int(lm.y*h)
                    lm_list.append([idx, cx, cy])
                        
                mp.drawing_utils.draw_landmarks(
                    image, hand_landmark, mp.hands.HAND_CONNECTIONS)
        
        # Detect handedness
        if results.multi_handedness:
            for idx, hand_handedness in enumerate(results.multi_handedness):
                handedness_dict = MessageToDict(hand_handedness)    
        
        # Detect finger count
        total_fingers = 0
        if len(lm_list) != 0:
            fingers = []
            for id in tip_id:
                
                # Thumb
                if id == tip_id[0]:
                    # When using front camera, the handedness is mirrored
                    if results.multi_handedness[0].classification[0].label == "Left":
                        if lm_list[id][1] > lm_list[id-1][1]:
                            fingers.append(1)
                            continue
                        else:
                            fingers.append(0)
                            continue
                    else:

                        if lm_list[id][1] < lm_list[id-1][1]:
                            fingers.append(1)
                            continue
                        else:
                            fingers.append(0)
                            continue
                # 4 fingers    
                if lm_list[id][2] < lm_list[id-2][2]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            total_fingers = fingers.count(1)
            

        # Display finger_image that is detected        
        h, w, c = overlay_list[total_fingers].shape
        image[0:h, 0:w] = overlay_list[total_fingers]
        
        cv2.rectangle(image, (20, 225), (140, 425), (0, 255, 0), cv2.FILLED)
        cv2.putText(image, str(total_fingers), (30, 375), cv2.FONT_HERSHEY_PLAIN, 
                                              10, (255, 0, 0), 25)
        
        # Display framerate
        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime
        cv2.putText(image, f"FPS: {int(fps)}", (500, 50), cv2.FONT_HERSHEY_PLAIN,
                    2, (255, 0, 0), 2)
    
        # Recolor for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        cv2.imshow("Hand detection", image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

<!-- ![image](https://user-images.githubusercontent.com/60457367/123659592-715aba80-d865-11eb-9acd-a15b719c2fe9.png) -->